<a href="https://colab.research.google.com/github/QuentasiJhonny/IA2/blob/main/RNN_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch.autograd import Variable
import numpy as np
import pandas as pd
import pylab as pl
import torch.nn.init as init

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:

filename = '/content/gdrive/MyDrive/datasets/cotizacionUFV/cotizacionUFV.xlsx'
data_training = pd.read_excel(filename, header=0)

print(data_training.shape)
print (data_training.head(10))

(273, 2)
       Fecha Cotizacion
0 2021-01-01    2,35844
1 2021-01-02    2,35837
2 2021-01-03    2,35830
3 2021-01-04    2,35823
4 2021-01-05    2,35816
5 2021-01-06    2,35809
6 2021-01-07    2,35802
7 2021-01-08    2,35795
8 2021-01-09    2,35788
9 2021-01-10    2,35781


In [27]:
data_train = data_training['Cotizacion'].values
print(data_train)



['2,35844' '2,35837' '2,35830' '2,35823' '2,35816' '2,35809' '2,35802'
 '2,35795' '2,35788' '2,35781' '2,35785' '2,35789' '2,35793' '2,35797'
 '2,35801' '2,35805' '2,35809' '2,35813' '2,35817' '2,35821' '2,35825'
 '2,35829' '2,35833' '2,35837' '2,35841' '2,35845' '2,35849' '2,35853'
 '2,35857' '2,35861' '2,35861' '2,35870' '2,35875' '2,35880' '2,35885'
 '2,35890' '2,35895' '2,35900' '2,35905' '2,35910' '2,35915' '2,35923'
 '2,35931' '2,35939' '2,35947' '2,35955' '2,35963' '2,35971' '2,35979'
 '2,35987' '2,35995' '2,36003' '2,36011' '2,36019' '2,36027' '2,36035'
 '2,36043' '2,36051' '2,36059' '2,36066' '2,36073' '2,36080' '2,36087'
 '2,36094' '2,36101' '2,36108' '2,36115' '2,36122' '2,36129' '2,36138'
 '2,36147' '2,36156' '2,36165' '2,36174' '2,36183' '2,36192' '2,36201'
 '2,36210' '2,36219' '2,36228' '2,36237' '2,36246' '2,36255' '2,36264'
 '2,36273' '2,36282' '2,36291' '2,36300' '2,36309' '2,36318' '2,36327'
 '2,36336' '2,36345' '2,36354' '2,36363' '2,36372' '2,36381' '2,36390'
 '2,36

In [28]:
import torch
from torch.utils.data import Dataset, DataLoader

In [29]:

data_train
X_train, y_train = data_train, data_train
X_valid, y_valid = data_train, data_train
X_test, y_test = data_train, data_train
X_train.shape,y_train.shape

((273,), (273,))

In [30]:
import torch
from torch.utils.data import Dataset, DataLoader

class CotizacionDataset(Dataset):
  def __init__(self, X, y=None, train=True):
    self.X = X
    self.y = y
    self.train = train

  def __len__(self):
    return len(self.X)

  def __getitem__(self, ix):
    if self.train:
      return torch.from_numpy(self.X[ix]), torch.from_numpy(self.y[ix])
    return torch.from_numpy(self.X[ix])

dataset = {
    'train': CotizacionDataset(X_train, y_train),
    'eval': CotizacionDataset(X_valid, y_valid),
    'test': CotizacionDataset(X_test, y_test, train=False)
}

dataloader = {
    'train': DataLoader(dataset['train'], shuffle=True, batch_size=64),
    'eval': DataLoader(dataset['eval'], shuffle=False, batch_size=64),
    'test': DataLoader(dataset['test'], shuffle=False, batch_size=64)
}

In [31]:
class MLP(torch.nn.Module):
  def __init__(self, n_in=30, n_out=30):
    super().__init__()
    self.fc = torch.nn.Linear(n_in, n_out)

  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

mlp = MLP()
mlp

MLP(
  (fc): Linear(in_features=30, out_features=30, bias=True)
)

In [32]:
class SimpleRNN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = torch.nn.RNN(input_size=1, hidden_size=1, num_layers=1, batch_first=True)

  def forward(self, x):
    x, h = self.rnn(x) 
    # solo queremos la última salida
    return x[:,-1]

rnn = SimpleRNN()

In [33]:
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"

def fit(model, dataloader, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.MSELoss()
    bar = tqdm(range(1, epochs+1))
    for epoch in bar:
        model.train()
        train_loss = []
        for batch in dataloader['train']:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        model.eval()
        eval_loss = []
        with torch.no_grad():
            for batch in dataloader['eval']:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                eval_loss.append(loss.item())
        bar.set_description(f"loss {np.mean(train_loss):.5f} val_loss {np.mean(eval_loss):.5f}")
        
def predict(model, dataloader):
    model.eval()
    with torch.no_grad():
        preds = torch.tensor([]).to(device)
        for batch in dataloader:
            X = batch
            X = X.to(device)
            pred = model(X)
            preds = torch.cat([preds, pred])
        return preds

In [34]:
fit(mlp, dataloader)

  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: ignored

In [1]:
y_pred = predict(mlp, dataloader['test'])
plot_series(X_test, y_test, y_pred.cpu().numpy())
mean_squared_error(y_test, y_pred.cpu())

NameError: ignored